In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from keras.models import load_model

DATA_PATH = '../input/aptos2019-blindness-detection/'

DIM_X = 256
DIM_Y = 256
BATCH_SIZE = 32


import cv2
import keras.backend as K


def crop_image_from_gray(img, tol=7):
    # If for some reason we only have two channels
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    # If we have a normal RGB images
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol

        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if (check_shape == 0):  # image is too dark so that we crop out everything,
            return img  # return original image
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img


def circle_crop_v2(img):
    height, width, depth = img.shape
    largest_side = np.max((height, width))
    img = cv2.resize(img, (largest_side, largest_side))

    height, width, depth = img.shape

    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)

    return img


def preprocess_image(image, sigmaX=25, DIM_X=256, DIM_Y=256):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = circle_crop_v2(image)
    image = cv2.resize(image, (DIM_X, DIM_Y))
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)
    return image


In [ ]:
class FixedDropout(tf.keras.layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape

        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

In [ ]:
# model = load_model('../input/resnet/resnet_06-12.h5', custom_objects={'kappa_keras': kappa_keras, 'kappa_loss': KAPPA_LOSS})
# model = load_model('../input/effnet/effnet_06-12.h5', custom_objects={'kappa_keras': kappa_keras, 'kappa_loss': KAPPA_LOSS, 'FixedDropout': FixedDropout})
model = load_model('../input/effnet-8/effnet_09-12_2.h5', custom_objects={'FixedDropout': FixedDropout})

In [ ]:
# import os
# import numpy as np
# import pandas as pd
# from PIL import Image
# from tqdm import tqdm

# import os
# from PIL import Image
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# STANDARDIZE_CROP_RATIO = 0.792
# OVERCROP_THRESHOLD = 25
# ZERO_TOLERANCE = 2
# BLUE_LAYER_IDX = 2
# RED_LAYER_IDX = 0


# def autocrop_scale(path, IMG_DIM=(512, 512), EPSILON=7, standardize_crop=False, return_crop_only=False):
#     '''
#     Loads the image file and automatically crops based on dark regions and rescales
#         to a specified dimension ignoring aspect ratio

#     :param path: str - path to file
#     :param IMG_DIM: tuple(int,int)/None - final dimension of image (no rescale if None)
#     :param EPSILON: int - minimum threshold for mean row/col pixel value
#     :param standardize_crop: bool - if True, intentionally over-crop perfect circles to standardize to
#                                     the images that are not perfect circles
#     :param return_crop_only: bool - if True, does not resize and return numpy array
#     :return: PIL.Image/np.ndarray
#     '''

#     # loads and convert images
# #     img = Image.open(path)
# #     data = np.asarray(img)
#     data = path
#     gray_data = data.mean(axis=2)

#     # dynamically determine non-dark regions based on grayscale mean of row/col values
#     limit_h = np.where(gray_data.mean(axis=0) >= EPSILON)[0]
#     horizontal = (limit_h[0], limit_h[-1])
#     limit_v = np.where(gray_data.mean(axis=1) >= EPSILON)[0]

#     # over-cropping
#     if standardize_crop and (abs((limit_h[-1] - limit_h[0]) - (limit_v[-1] - limit_v[0])) <= OVERCROP_THRESHOLD):
#         # executes only if the limits are approximately square
#         crop_v = STANDARDIZE_CROP_RATIO * (limit_v[-1] - limit_v[0]) / 2
#         center = (limit_v[-1] + limit_v[0]) / 2
#         vertical = (int(center - crop_v), int(center + crop_v))
#     else:
#         vertical = (limit_v[0], limit_v[-1])

#     # crops array representation
#     new_data = data[vertical[0]:vertical[1] + 1, horizontal[0]:horizontal[1] + 1, :]

#     # reverts to original if too much cropped
#     if new_data.shape[0] < 100 or new_data.shape[1] < 100:
#         new_data = data

#     if return_crop_only:
#         return new_data

#     # converts and resize to PIL image
#     processed_img = Image.fromarray(new_data)
#     if IMG_DIM is not None:
#         processed_img = processed_img.resize(IMG_DIM)

#     return processed_img


# def standard_crop(path, IMG_DIM=(512, 512), ratio=4 / 3, cratio=592 / 386, contrast_fnc=None, **kwargs):
#     '''
#     Crops an image to an approximate form found in majority of the images in the test dataset.

#     The values found here is derived from manual measurements with trigonometry.

#     :param path: str - path of image
#     :param IMG_DIM: tuple(int, int)/None - final dimension of image (no rescale if None)
#     :param ratio: float - final aspect ratio of output image
#     :param cratio: flaot - ratio of chord at the bottom to the radius
#     :param contrast_fnc: None/function - contrast function
#     :return: PIL.Image
#     '''
# #     data = autocrop_scale(path, return_crop_only=True)  # crops away all excess background
#     data = crop_image_from_gray(path, tol=7)
# #     data = circle_crop_v2(path)
    
#     h, l = data.shape[0], data.shape[1]

#     # dynamically determine the radius of the circle
#     sample_column = data[:, 1, :]  # second column of data
#     # first non-background pixel
#     edge = np.where((data.mean(axis=2)[:, 0] > sample_column.mean() + 5 / sample_column.mean(axis=1).std()))
#     edge = edge[0][edge[0] > h / 8]  # accepts if not near the corner
#     if len(edge) > 0:
#         r = np.sqrt((edge[0] - h / 2) ** 2 + (l / 2) ** 2)
#     else:
#         r = l / 2  # assumes length is the diameter

#     delta_h = r - np.sqrt(r ** 2 - (cratio * r / 2) ** 2)  # height to be cropped away at both the top and bottom
#     # max(..., 0) is needed to handle the cases when the specified image is a perfect circle and cropped circle
#     data = data[max(int(delta_h - (2 * r - h) / 2), 0):h - max(int(delta_h - (2 * r - h) / 2), 0), ...]

#     h, l = data.shape[0], data.shape[1]
#     delta_l = l - h * ratio  # length to be cropped away at both the left and right

#     data = data[:, int(delta_l / 2):l - int(delta_l / 2), :]

#     data = cv2.resize(data, dsize=IMG_DIM, interpolation=cv2.INTER_AREA)

#     if contrast_fnc is not None:
#         data = contrast_fnc(data, **kwargs)

# #     processed_img = Image.fromarray(data)

#     return data.reshape(*data.shape)

# def contrast_enhance(img, sigma=10, gray=False):
#     '''
#     Contrast technique based on Kaggle notebook

#     :param img: np.ndarray - image
#     :param sigma: int - degree of contrast (affects efficiency of code)
#     :param gray: bool - returns grayscale (i.e. 1 channel)
#     :return: nd.ndarray
#     '''
#     if gray:
#         img = img.mean(axis=2)
#         img = np.dstack((img, img, img)).astype(np.uint8)

#     return cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), sigma), -4, 128)


In [ ]:
submission_df = pd.read_csv(DATA_PATH + '/test.csv')
submission_df['filename'] = submission_df['id_code'].astype(str)+'.png'

submission = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1/255.)
# submission = ImageDataGenerator(preprocessing_function=lambda x: standard_crop(x,IMG_DIM=(256,256), contrast_fnc=contrast_enhance, gray=False), rescale=1/255.)
gen = submission.flow_from_dataframe(dataframe = submission_df,
                                       directory= DATA_PATH + "test_images",
                                       x_col="filename", 
                                       batch_size = 64,
                                       shuffle=False,
                                       class_mode=None, 
                                       target_size=(256, 256), validate_filenames=False)

pred = model.predict_generator(gen, verbose=1)




In [ ]:
pred = np.argmax(pred, axis=1)

results = dict(zip(gen.filenames, pred))
labels = list(map(lambda x: results[x], submission_df['filename']))

submission_df.drop(columns=['filename'], inplace= True)
submission_df['diagnosis'] = labels
submission_df.to_csv('submission.csv', index=False)

In [ ]:
from collections import Counter
cnt = Counter(labels)
cnt